#### Prepare notebook with relevant modules

In [173]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [174]:
from odp.client import OdpClient # The SDK

from shapely.geometry import box

import pandas as pd
import matplotlib.pyplot as plt
import pydeck as pdk

In [175]:
def create_h3_pydeck_map(df, hex_column, value_column, scale_factor=500, center_lat=58.0, center_lon=10.0, zoom=5):
    return pdk.Deck(
        layers=[pdk.Layer( 'H3HexagonLayer', df, get_hexagon=hex_column, get_fill_color=f'[254, 119, 76, {scale_factor} * {value_column}]', opacity=1, extruded=False,
            get_line_color=[32, 10, 58, 10], line_width_min_pixels=1, pickable=True, filled=True, stroked=True )],
        initial_view_state=pdk.ViewState(latitude=center_lat, longitude=center_lon, zoom=zoom, pitch=0), map_provider="mapbox", map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'})

In [176]:
base_url = "https://odcat.dev.hubocean.io"
client = OdpClient(base_url=base_url)

In [177]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("7af5bc0f-c12a-451b-9834-fa6c4e0a2c87"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

'Ocean Sensitive Areas - version 1.0.0 - table'

In [40]:
client = OdpClient()


In [41]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("f720eb09-6c6f-459b-a469-2eaa4c46bb7a"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

HTTPError: 403 Client Error: Forbidden for url: https://api.hubocean.earth/api/table/v2/select?table_id=f720eb09-6c6f-459b-a469-2eaa4c46bb7a&tx_id=

In [43]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("468bef0c-5934-44e6-bd3e-5a60a0b326b6"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

HTTPError: 403 Client Error: Forbidden for url: https://api.hubocean.earth/api/table/v2/select?table_id=468bef0c-5934-44e6-bd3e-5a60a0b326b6&tx_id=

In [178]:
next(osa_data.select().dataframes())

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
0,0.000,1.000,0.0,86e004827ffffff,0.0,0.0,0.0,"POLYGON ((61.775 -60.4656, 61.8003 -60.4333, 6...",0.0,1.000,0.000,0.000
1,0.000,1.000,0.0,86e00482fffffff,0.0,0.0,0.0,"POLYGON ((61.6686 -60.4448, 61.694 -60.4125, 6...",0.0,1.000,0.000,0.000
2,0.000,1.000,0.0,86e004837ffffff,0.0,0.0,0.0,"POLYGON ((61.8661 -60.4274, 61.8913 -60.3951, ...",0.0,1.000,0.000,0.000
3,0.693,0.500,0.0,86e004867ffffff,0.0,0.0,0.0,"POLYGON ((61.4711 -60.4618, 61.4967 -60.4296, ...",0.0,2.428,0.087,0.014
4,0.693,0.500,0.0,86e00486fffffff,0.0,0.0,0.0,"POLYGON ((61.3652 -60.4408, 61.3909 -60.4086, ...",0.0,2.428,0.087,0.014
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000,1.000,0.0,86e10018fffffff,0.0,0.0,0.0,"POLYGON ((68.9432 -65.7561, 68.9655 -65.7221, ...",0.0,1.000,0.000,0.000
1996,0.693,0.500,0.0,86e100197ffffff,0.0,0.0,0.0,"POLYGON ((69.184 -65.7299, 69.2059 -65.6958, 6...",0.0,2.428,0.087,0.014
1997,1.099,0.333,0.0,86e10019fffffff,0.0,0.0,0.0,"POLYGON ((69.0456 -65.7133, 69.0677 -65.6793, ...",0.0,3.264,0.137,0.023
1998,1.099,0.333,0.0,86e1001a7ffffff,0.0,0.0,0.0,"POLYGON ((69.118 -65.8322, 69.1401 -65.7981, 6...",0.0,3.264,0.137,0.023


In [179]:
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12

In [180]:
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 18
lat_max = 21
lon_min = -77
lon_max = -72

In [181]:
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((-72 18, -72 21, -77 21, -77 18, -72 18))'

In [183]:
df = osa_data.select(f'geometry within "{query_geometry}"').dataframe()

In [184]:
df.tail(2)

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
1524,0.0,0.0,0.0,866736d6fffffff,1.0,0.0,0.0,"POLYGON ((-76.7583 18.9219, -76.7856 18.9072, ...",0.0,17.5,0.0,0.167
1525,0.0,0.0,0.0,866736d77ffffff,1.0,0.0,0.0,"POLYGON ((-76.8435 18.9057, -76.8162 18.9205, ...",0.0,17.5,0.0,0.167


In [187]:
# Vizualize data sample for the Shannon index. You can swap out shannon_norm for any other attribute e.g. seagrass
map_viz = create_h3_pydeck_map(df, 'hex6', 'simpson', scale_factor=500, center_lat=((lat_min + lat_max) / 2), center_lon=((lon_min + lon_max) / 2), zoom=6)
map_viz

{
  "initialViewState": {
    "latitude": 19.5,
    "longitude": -74.5,
    "pitch": 0,
    "zoom": 6
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-76.8328 19.9584, -76.8052 19.973, -76.8017 20.0049, -76.8259 20.0221, -76.8535 20.0075, -76.857 19.9757, -76.8328 19.9584))",
          "hex6": "864590537ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 1.0
        },
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-76.9262 20.5405, -76.8985 20.555, -76.895 20.5869, -76.9193 20.6043, -76.9471 20.5898, -76.9506 20.5579, -76.9262 20.5405))",
          "hex6": "86459061fffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 1.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-76.8949 19.8656, -76.8673 19.8802, -76.8639 19.912, -76.888 19.9292, -76.9157 19.9146, -76.9191 19.8828, -76.8949 19.8656))",
          "hex6": "864590c87ffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-76.8639 19.912, -76.8363 19.9266, -76.8328 19.9584, -76.857 19.9757, -76.8846 19.9611, -76.888 19.9292, -76.8639 19.912))",
          "hex6": "864590c9fffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 34.0,
          "S_raw": 0.333,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-76.9467 19.8682, -76.9191 19.8828, -76.9157 19.9146, -76.9398 19.9319, -76.9675 19.9172, -76.9709 19.8854, -76.9467 19.8682))",
          "hex6": "864590cafffffff",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 0.0,
          "shannon_norm": 0.0,
          "simpson": 0.0
        },
        {
          "S": 3.264,
          "S_raw": 0.023,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-76.9675 19.9172, -76.9398 19.9319, -76.9364 19.9637, -76.9606 19.9809, -76.9883 19.9663, -76.9917 19.9344, -76.9675 19.9172))",
          "hex6": "864590cf7ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 1.099,
          "shannon_norm": 0.137,
          "simpson": 0.333
        },
        {
          "S": 4.927,
          "S_raw": 0.04,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-76.0075 20.3915, -75.9801 20.4058, -75.9764 20.4374, -76.0002 20.4548, -76.0276 20.4405, -76.0313 20.4088, -76.0075 20.3915))",
          "hex6": "864592547ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 1.906,
          "shannon_norm": 0.238,
          "simpson": 0.156
        },
        {
          "S": 1.0,
          "S_raw": 0.0,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-76.1378 20.8301, -76.1103 20.8443, -76.1066 20.876, -76.1306 20.8935, -76.1581 20.8793, -76.1618 20.8476, -76.1378 20.8301))",
          "hex6": "864592667ffffff",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "

In [ ]:
list(osa_data.select('geometry contains "POINT(128.38 0.39)"').rows())

In [ ]:
next(osa_data.select('geometry contains "POINT(128.38 0.39)"').dataframes())